## For gradient boosting

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import roc_auc_score, f1_score, recall_score, accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import numpy as np


In [2]:
## 1. Chargement des données
df = pd.read_csv('celldata.csv')

# Séparer la cible (Y) et les variables explicatives (X)
X = df.drop('Churn', axis=1)
y = df['Churn']

## 2. Définition des Catégories de Variables
# Variables numériques continues 
numerical_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'Salary']

# Variables catégorielles nominales 
categorical_features = ['Geography', 'Gender']

# Variables binaires déjà bianires 
binary_features = ['HasCrCard', 'IsActiveMember']


## 3. Création du Préprocesseur 

preprocessor = ColumnTransformer(
    transformers=[
        # 1. Standardisation pour les variables numériques
        ('num', StandardScaler(), numerical_features),
        
        # 2. Encodage One-Hot pour les variables catégorielles
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        
        # 3. Passer les variables binaires directement
        ('bin', 'passthrough', binary_features)
    ],
    remainder='drop' 
)

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [3]:
# Création du Pipeline
# Utilise la technique de gradient boosting pour construire une forêt d'arbres.
gbm_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', GradientBoostingClassifier(
                                   n_estimators=100, 
                                   learning_rate=0.1, 
                                   max_depth=3, 
                                   random_state=42))])

# Entraînement
gbm_pipeline.fit(X_train, y_train)

# Prédiction
y_pred_gbm = gbm_pipeline.predict(X_test)
y_proba_gbm = gbm_pipeline.predict_proba(X_test)[:, 1]

In [7]:
from sklearn.metrics import recall_score, precision_score
import pandas as pd
import numpy as np

# --- 1. Préparation des Données pour l'Analyse d'Équité ---

# X_test est la DataFrame non transformée utilisée pour la prédiction
gender_test = X_test['Gender'].reset_index(drop=True)

# y_test et y_pred_gbm (prédictions binaires du Gradient Boosting) sont utilisés ici
equity_df = pd.DataFrame({
    'Gender': gender_test,
    'True_Churn': y_test.reset_index(drop=True),
    'Predicted_Churn': y_pred_gbm 
})

results_female = equity_df[equity_df['Gender'] == 'Female']
results_male = equity_df[equity_df['Gender'] == 'Male']

# --- 2. Fonctions de Calcul des Taux ---

def calculate_rates(data):
    """Calcule le taux de Churn prédit, le Recall et la Precision pour un sous-groupe."""
    
    # Évite les erreurs si le sous-groupe est vide
    if data.empty:
        return {'Pred_Rate': np.nan, 'Recall': np.nan, 'Precision': np.nan}
    
    # Taux de prédiction de Churn (pour l'Indépendance)
    pred_rate = data['Predicted_Churn'].mean() 
    
    # Recall (pour la Séparation)
    # Gère le cas où il n'y a pas de Churn réel dans le groupe (évite division par zéro)
    if data['True_Churn'].sum() > 0:
        recall = recall_score(data['True_Churn'], data['Predicted_Churn'])
    else:
        recall = np.nan
        
    # Precision (pour la Suffisance)
    # Gère le cas où il n'y a pas de Churn prédit dans le groupe
    if data['Predicted_Churn'].sum() > 0:
        precision = precision_score(data['True_Churn'], data['Predicted_Churn'])
    else:
        precision = np.nan
        
    return {'Pred_Rate': pred_rate, 'Recall': recall, 'Precision': precision}

# Calcul des métriques pour chaque groupe
metrics_female = calculate_rates(results_female)
metrics_male = calculate_rates(results_male)

In [8]:
print("=====================================================")
print("=== RÉSULTATS DE L'ANALYSE D'ÉQUITÉ (Gender) - GBM ===")
print("=====================================================")

# --- Affichage des Métriques de Base ---
print(f"Nombre de Femmes : {len(results_female)}")
print(f"Nombre d'Hommes : {len(results_male)}")
print("-" * 55)


# --- A. Critère d'Indépendance (Independence) - P(Ŷ=1|A) ---
print("A. CRITÈRE D'INDÉPENDANCE (Parité Démographique)")
print("  Taux de prédiction de Churn (P(Ŷ=1|A)) :")
print(f"  > Femmes : {metrics_female['Pred_Rate']:.4f}")
print(f"  > Hommes : {metrics_male['Pred_Rate']:.4f}")

diff_pred_rate = abs(metrics_female['Pred_Rate'] - metrics_male['Pred_Rate'])

print(f"  DIFFÉRENCE ABSOLUE : {diff_pred_rate:.4f}")

if diff_pred_rate < 0.05: # Utilisation d'un seuil arbitraire de 5% pour l'évaluation
    print("  -> INTERPRÉTATION : Le critère est RAPPROCHÉ. Le modèle prédit des taux de Churn similaires.")
else:
    print("  -> INTERPRÉTATION : Le critère est VIOLÉ. Le modèle cible un genre plus que l'autre.")

print("-" * 55)


# --- B. Critère de Séparation (Separation) - P(Ŷ=1|Y=1, A) ---
print("B. CRITÈRE DE SÉPARATION (Égalité des Chances / Recall)")
print("  Rappel (Recall) - Taux de détection du Churn réel :")
print(f"  > Femmes : {metrics_female['Recall']:.4f}")
print(f"  > Hommes : {metrics_male['Recall']:.4f}")

diff_recall = abs(metrics_female['Recall'] - metrics_male['Recall'])

print(f"  DIFFÉRENCE ABSOLUE : {diff_recall:.4f}")

if diff_recall < 0.10: # Utilisation d'un seuil arbitraire de 10% pour l'évaluation
    print("  -> INTERPRÉTATION : Le critère est RAPPROCHÉ. Le modèle est également bon pour détecter le Churn dans les deux groupes.")
elif metrics_female['Recall'] < metrics_male['Recall']:
    print("  -> INTERPRÉTATION : Le critère est VIOLÉ. Le modèle est moins performant pour détecter le Churn chez les Femmes.")
else:
    print("  -> INTERPRÉTATION : Le critère est VIOLÉ. Le modèle est moins performant pour détecter le Churn chez les Hommes.")

print("-" * 55)


# --- C. Critère de Suffisance (Sufficiency) - P(Y=1|Ŷ=1, A) ---
print("C. CRITÈRE DE SUFFISANCE (Égalité des Prédictions / Precision)")
print("  Précision (Precision) - Fiabilité des prédictions de Churn :")
print(f"  > Femmes : {metrics_female['Precision']:.4f}")
print(f"  > Hommes : {metrics_male['Precision']:.4f}")

diff_precision = abs(metrics_female['Precision'] - metrics_male['Precision'])

print(f"  DIFFÉRENCE ABSOLUE : {diff_precision:.4f}")

if diff_precision < 0.10: # Utilisation d'un seuil arbitraire de 10% pour l'évaluation
    print("  -> INTERPRÉTATION : Le critère est RAPPROCHÉ. Les prédictions de Churn sont d'une fiabilité similaire pour les deux groupes.")
elif metrics_female['Precision'] < metrics_male['Precision']:
    print("  -> INTERPRÉTATION : Le critère est VIOLÉ. Les prédictions de Churn pour les Femmes sont moins fiables (plus de faux positifs).")
else:
    print("  -> INTERPRÉTATION : Le critère est VIOLÉ. Les prédictions de Churn pour les Hommes sont moins fiables (plus de faux positifs).")

print("=====================================================")

=== RÉSULTATS DE L'ANALYSE D'ÉQUITÉ (Gender) - GBM ===
Nombre de Femmes : 1064
Nombre d'Hommes : 1336
-------------------------------------------------------
A. CRITÈRE D'INDÉPENDANCE (Parité Démographique)
  Taux de prédiction de Churn (P(Ŷ=1|A)) :
  > Femmes : 0.1438
  > Hommes : 0.0921
  DIFFÉRENCE ABSOLUE : 0.0517
  -> INTERPRÉTATION : Le critère est VIOLÉ. Le modèle cible un genre plus que l'autre.
-------------------------------------------------------
B. CRITÈRE DE SÉPARATION (Égalité des Chances / Recall)
  Rappel (Recall) - Taux de détection du Churn réel :
  > Femmes : 0.4585
  > Hommes : 0.4563
  DIFFÉRENCE ABSOLUE : 0.0022
  -> INTERPRÉTATION : Le critère est RAPPROCHÉ. Le modèle est également bon pour détecter le Churn dans les deux groupes.
-------------------------------------------------------
C. CRITÈRE DE SUFFISANCE (Égalité des Prédictions / Precision)
  Précision (Precision) - Fiabilité des prédictions de Churn :
  > Femmes : 0.8301
  > Hommes : 0.7642
  DIFFÉRENCE